# Feature Extraction

In [5]:
# !pip install transformers einops timm pillow

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("datasetTeste.csv")

In [3]:
df.head()

,movieId,title,genres,imdbId,tmdbId,imageLink
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://image.tmdb.org/t/p/w500/uXDfjJbdP4ijW5...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,https://image.tmdb.org/t/p/w500/vgpXmVaVyUL7GG...
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,https://image.tmdb.org/t/p/w500/1FSXpj5e8l4KH6...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,https://image.tmdb.org/t/p/w500/qJU6rfil5xLVb5...
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,https://image.tmdb.org/t/p/w500/rj4LBtwQ0uGrpB...


In [4]:
from transformers import AutoModel
from PIL import Image
import requests

C:\Users\SmartHint\Documents\INFOESTE\2024\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = AutoModel.from_pretrained('jinaai/jina-clip-v1', trust_remote_code=True)

In [6]:
image_embeddings = model.encode_image(Image.open(requests.get("https://image.tmdb.org/t/p/w500/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg", stream=True).raw))

In [7]:
image_embeddings.shape

(768,)

# ElasticSearch

In [14]:
# !pip install elasticsearch

In [4]:
from elasticsearch import Elasticsearch

In [5]:
client = Elasticsearch(
    "http://localhost:9200"
)

In [25]:
mapping = {
    "mappings": {
        "properties": {
            "title":{
                "type":"keyword"
            },
            "image":{
                "type":"text"
            },
            "image_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": "true",
                "similarity": "cosine",
            }
        }
    }
}

In [28]:
client.indices.create(index='teste_movies', ignore=400, body=mapping)


C:\Users\SmartHint\AppData\Local\Temp\ipykernel_42616\2682560762.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.create(index='teste_movies', ignore=400, body=mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'teste_movies'})

In [6]:
index_name = "teste_movies"

In [29]:
for i,row in df.iterrows():
    doc = {}
    vector = model.encode_image(Image.open(requests.get(row["imageLink"], stream=True).raw))
    doc["title"] = row["title"]
    doc["image"] = row["imageLink"]
    doc["image_vector"] = vector
    client.index(index=index_name, id = row["title"],document= doc)

In [24]:
vector.tolist()

[0.003021813230589032,
 -0.047413937747478485,
 -0.05115253105759621,
 -0.023678036406636238,
 0.03265189006924629,
 0.0706104263663292,
 -0.05430102348327637,
 -0.006592853926122189,
 0.006772516295313835,
 -0.03459169343113899,
 0.02675323374569416,
 -0.02727280929684639,
 -0.07542948424816132,
 0.03751562163233757,
 0.04048704355955124,
 0.0023599970154464245,
 0.04530247300863266,
 -0.027062145993113518,
 -0.0811946764588356,
 0.06939052790403366,
 -0.052984632551670074,
 0.06662893295288086,
 0.005979945883154869,
 -0.014677315019071102,
 0.011179769411683083,
 0.032226771116256714,
 0.08100070059299469,
 0.016204196959733963,
 -0.004103426821529865,
 0.14485973119735718,
 0.039421264082193375,
 0.03720790147781372,
 -0.028231333941221237,
 0.01727282628417015,
 0.02143106423318386,
 -0.05551023781299591,
 0.022094715386629105,
 0.02261224389076233,
 -0.003267137799412012,
 0.009867338463664055,
 -0.03920464217662811,
 0.018606361001729965,
 0.0075196437537670135,
 -0.017812129110

In [10]:
image_embeddings = model.encode_image(Image.open(requests.get("https://image.tmdb.org/t/p/w500/2MFIhZAW0CVlEQrFyqwa4U6zqJP.jpg", stream=True).raw))

In [11]:
doc = {}
# vector = model.encode_image(Image.open(requests.get(row["imageLink"], stream=True).raw))
doc["title"] = "Toy Story 2"
doc["image"] = "https://image.tmdb.org/t/p/w500/2MFIhZAW0CVlEQrFyqwa4U6zqJP.jpg"
doc["image_vector"] = image_embeddings
client.index(index=index_name, id = "Toy Story 2",document= doc)

ObjectApiResponse({'_index': 'teste_movies', '_id': 'Toy Story 2', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 625, '_primary_term': 2})